In [2]:
import os

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
from scipy.stats import multivariate_normal
from sklearn import svm
from sklearn.decomposition import PCA

**1. Support Vector Machine**


In [ ]:
data = pd.read_csv("./x_train.csv", header=None) / 255
label = pd.read_csv("./t_train.csv", header=None)


**2. Gaussian Mixture Model**


In [ ]:
image = cv2.imread("./hw3.jpg")